<a href="https://colab.research.google.com/github/OnlyBelter/machine-learning-note/blob/master/pytorch/pytorch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:


*   https://pytorch.org/tutorials/beginner/basics/intro.html
*   https://youtu.be/OMDn66kM9Qc?si=p_CggEi2Rvhv6NKK

*   The lines with the comment "<-- here" need to be addressed to enable GPU usage.






In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [2]:
device = (
"cuda"
if torch.cuda.is_available()
else "mps"
if torch.backends.mps.is_available()
else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)  # <-- here
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
learning_rate = 0.001
batch_size = 64
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size=64):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        ## Compute prediction and loss

        X = X.to(device)  # <-- here
        y = y.to(device)  # <-- here

        # 1 forward
        pred = model(X)

        # 2 compute the objective function
        loss = loss_fn(pred, y)

        # 3 cleaning the gradients
        optimizer.zero_grad()

        ## Backpropagation
        # 4 accumulate the partial derivatives of loss wrt params
        loss.backward() # params.grad._sum(dloss/dparams)

        # 5 step in the opposite direction of the gradient
        optimizer.step() # with torch.no_grad(): params = params - eta * params.grad

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)  # <-- here
            y = y.to(device)  # <-- here
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.88MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 6.29MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314119  [   64/60000]
loss: 2.295859  [ 6464/60000]
loss: 2.271147  [12864/60000]
loss: 2.256501  [19264/60000]
loss: 2.249931  [25664/60000]
loss: 2.224010  [32064/60000]
loss: 2.229151  [38464/60000]
loss: 2.202724  [44864/60000]
loss: 2.191143  [51264/60000]
loss: 2.152983  [57664/60000]
Test Error: 
 Accuracy: 41.9%, Avg loss: 2.150848 

Epoch 2
-------------------------------
loss: 2.163958  [   64/60000]
loss: 2.150249  [ 6464/60000]
loss: 2.088012  [12864/60000]
loss: 2.097677  [19264/60000]
loss: 2.054755  [25664/60000]
loss: 1.995352  [32064/60000]
loss: 2.021057  [38464/60000]
loss: 1.945333  [44864/60000]
loss: 1.952094  [51264/60000]
loss: 1.865209  [57664/60000]
Test Error: 
 Accuracy: 55.7%, Avg loss: 1.870431 

Epoch 3
-------------------------------
loss: 1.906721  [   64/60000]
loss: 1.874307  [ 6464/60000]
loss: 1.754199  [12864/60000]
loss: 1.788075  [19264/60000]
loss: 1.684679  [25664/60000]
loss: 1.636643  [32064/600